# Fine-Tuning GPT2 on Colab GPU… For Free!

This is a colab notebook for the [associated Medium article](https://medium.com/p/340468c92ed)

## Installing Dependencies
We would run pip3 install transformers normally in Bash, but because this is in Colab, we have to run it with !

In [12]:
# !pip3 install transformers
!python -m pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-291usg2k
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-291usg2k
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-3.4.0-cp36-none-any.whl size=1275545 sha256=0de7fc7e37d43cfab7a9e0fe8a92f8e642015b8d7617294d8e43781fc8cba474
  Stored in directory: /tmp/pip-ephem-wheel-cache-1bq_jb7z/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
Successfully built transformers


## Getting WikiText Data

You can read more about WikiText data here. Overall, there's WikiText-2 and WikiText-103. We're going to use WikiText-2 because it's smaller, and we have limits in terms of how long we can run on GPU, and how much data we can load into memory in Colab. To download and run

In [3]:
%%bash
wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
unzip wikitext-2-raw-v1.zip

Archive:  wikitext-2-raw-v1.zip


--2020-10-24 19:48:20--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.16.14
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.16.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4721645 (4.5M) [application/zip]
Saving to: ‘wikitext-2-raw-v1.zip.1’

     0K .......... .......... .......... .......... ..........  1%  814K 6s
    50K .......... .......... .......... .......... ..........  2%  812K 6s
   100K .......... .......... .......... .......... ..........  3%  818K 5s
   150K .......... .......... .......... .......... ..........  4% 43.4M 4s
   200K .......... .......... .......... .......... ..........  5%  830K 4s
   250K .......... .......... .......... .......... ..........  6% 91.7M 4s
   300K .......... .......... .......... .......... ..........  7% 71.6M 3s
   350K .......... .......... .......... .......... ..........  8%  143M 3s
   400K .......... .

## Fine-Tuning GPT2

HuggingFace actually provides a script to help fine tune models here. We can just download the script by running

In [4]:
! wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py

--2020-10-24 19:48:51--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13076 (13K) [text/plain]
Saving to: ‘run_language_modeling.py’

run_language_modeli 100%[===================>]  12.77K  --.-KB/s    in 0s      

2020-10-24 19:48:51 (138 MB/s) - ‘run_language_modeling.py’ saved [13076/13076]



Now we are ready to fine tune.

There are many parameters to the script, and you can understand them by reading the manual. I'm just going to go over the important ones for basic training.

- `output_dir` is where the model will be output
- `model_type` is what model you want to use. In our case, it's gpt2 
- `model_name_or_path` is the path to the model. If you want to train from scratch, you can leave this blank. In our case, it's also gpt2 
- `do_train` tells it to train
- `train_data_file` points to the training file
- `do_eval` tells it to evaluate afterwards. Not always required, but good to have
- `eval_data_file` points to the evaluation file

Some extra ones you MAY care about, but you can also skip this.
- `save_steps` is when to save checkpoints. If you have limited memory, you can set this to -1 so it'll skip saving until the end
- `per_gpu_train_batch_size` is batch size for GPU. You can increase this if your GPU has enough memory. To be safe, you can start with 1 and ramp it up if you still have memory
- `num_train_epochs` is the number of epochs to train. Since we're fine-tuning, I'm going to set this to 2


In [3]:
# Install any package dependencies
!pip install datasets

# Import any required libraries
from datasets import load_dataset
from google.colab import drive
import os

# Use google drive to store initial dataset before caching (if required by the dataset):
drive.mount('/content/drive')

# load all other datasets here...
newsroom = load_dataset('newsroom', split='train', data_dir='/content/drive/My Drive/newsroom')
newsroom_test = load_dataset('newsroom', split='test', data_dir='/content/drive/My Drive/newsroom')
dataset_keys = {
  'newsroom_train': {
    'DATASET_OBJ': newsroom,
    'TEXT': 'text',
    'SMRY': 'summary'
  },
  'newsroom_test': {
    'DATASET_OBJ': newsroom_test,
    'TEXT': 'text',
    'SMRY': 'summary'
  }
}


def make_dataset_file(split='train', demo_mode=False):
  assert split in ['train', 'test']
  # Change <demo_mode> to False if you want to process each ENTIRE dataset 

  if demo_mode:
      print('DEMO MODE ACTIVE\n\n')

  processed_datasets = { }

  for name in dataset_keys.keys():
      # access respective keys to account for naming conventions
      ds_keys = dataset_keys[name]
      ds_obj = ds_keys['DATASET_OBJ']
      
      f = open('%s.txt' % name, 'w+')
      for index, entry in enumerate(ds_obj):
          # Get text and prepare for processing
          text = entry[ds_keys['TEXT']]
          text = ''.join(text.split("\n"))
          smry = entry[ds_keys['SMRY']]
          assert type(text) == str and type(smry) == str
          # concatenate break indicators and actual text/summaries
          f.write(''.join(['[TEXT]\n', text, '\n[SUMM]\n', smry, '[SEP]\n\n']))
          if index > 200:
            break
          if not index % 100000:
              print('Processing Dataset \"%s\" at Index %d' % (name, index))
              if demo_mode and index == 2000:
                  break
      print('\nFINISHED Processing Dataset \"%s\"\n' % name)
      f.close()

make_dataset_file()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using custom data configuration default
Reusing dataset newsroom (/root/.cache/huggingface/datasets/newsroom/default/1.0.0/4b405ccd64e15f685065870ea563a1e6a034d1bd269a5427f40146d81549095e)
Using custom data configuration default
Reusing dataset newsroom (/root/.cache/huggingface/datasets/newsroom/default/1.0.0/4b405ccd64e15f685065870ea563a1e6a034d1bd269a5427f40146d81549095e)


Processing Dataset "newsroom_train" at Index 0

FINISHED Processing Dataset "newsroom_train"

Processing Dataset "newsroom_test" at Index 0

FINISHED Processing Dataset "newsroom_test"



In [23]:
%%bash
export TRAIN_FILE=newsroom_train.txt
export TEST_FILE=newsroom_test.txt
export MODEL_NAME="yuvraj/summarizer-cnndm"
export OUTPUT_DIR=output_mine

python run_language_modeling.py \
    --output_dir=$OUTPUT_DIR \
    --model_type=$MODEL_NAME \
    --model_name_or_path=$MODEL_NAME \
    --do_train \
    --train_data_file=$TRAIN_FILE \
    --do_eval \
    --eval_data_file=$TEST_FILE \
    --per_gpu_train_batch_size=1 \
    --save_steps=-1 \
    --num_train_epochs=2 \


2020-10-24 20:15:33.623236: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
10/24/2020 20:15:35 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/24/2020 20:15:35 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='output_mine', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=1, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Oct24_20-15-35_6c1e22b6e772', logging_first_ste

## Results

To use it, you can run something like

In [7]:
from transformers import AutoTokenizer, AutoModelWithLMHead, pipeline
import torch
import numpy as np

OUTPUT_DIR = "./output_mine"
device = 'cpu'

# tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
# model = AutoModelWithLMHead.from_pretrained(OUTPUT_DIR)

tokenizer = AutoTokenizer.from_pretrained("yuvraj/summarizer-cnndm") 
model = AutoModelWithLMHead.from_pretrained("yuvraj/summarizer-cnndm")

# model = model.to(device)
model = model.to(device)

# summarizer = pipeline('summarization', model=model, tokenizer=tokenizer)
summarizer = pipeline('summarization', model=model, tokenizer=tokenizer)
                                        
# def generate(input_str, model=model, tokenizer=tokenizer, length=250, n=5):
#   output_text = input_str
#   summarizer = pipeline('summarization', model=model, tokenizer=tokenizer)
#   for word_num in range(length):
#     data = unmasker(output_text)
#     probs = []
#     current_prob = 0
#     for word in data:
#       current_prob += word['score']
#       probs.append((current_prob, word['token_str']))
#     choice = np.random.uniform(high=current_prob)
#     probs.append(choice, "")
#     probs = sorted(probs)
#     index = probs.index((choice, ""))
#     output_text = output_text + probs[index-1][1] + " "
#   return output_text

# generated_text = generate(" = University of Illinois = \n")
# print(generated_text)

text = newsroom[0]['text'].split(' ')
text = ' '.join(text[:691])

print(text + "\n")
print(summarizer(text))

print("\n\n")

# summarizer(text)

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:825: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


HAMBURG, Germany, June 3  As he left the soccer field after a club match in the eastern German city of Halle on March 25, the Nigerian forward Adebowale Ogungbure was spit upon, jeered with racial remarks and mocked with monkey noises. In rebuke, he placed two fingers under his nose to simulate a Hitler mustache and thrust his arm in a Nazi salute.

Marc Zoro, right, an Ivory Coast native, was a target of racial slurs from the home fans in Messina, Italy. Adriano, a star with Inter Milan, tried to persuade him to stay on the field.

From now until its conclusion on July 9, Jeff Z. Klein and other staff members of The Times and International Herald Tribune will track the world's most popular sporting event.

Your guide to the games in Germany: teams, rosters, schedules, statistics, venues and more.

In April, the American defender Oguchi Onyewu, playing for his professional club team in Belgium, dismissively gestured toward fans who were making simian chants at him. Then, as he went to

## Compressing/Zipping Model


In order for us to preserve this model, we should compress it and save it somewhere. This can be done easily with

In [ ]:
! tar -czf gpt2-tuned.tar.gz output/

which creates a file called `gpt2-tuned.tar.gz`

## Saving it to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Now you can copy your output model to your Google Drive by running

In [ ]:
!cp gpt2-tuned.tar.gz /content/drive/My\ Drive/